In [ ]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sb
from gprofiler import GProfiler
import seaborn as sns
import rpy2.rinterface_lib.callbacks
import logging

from rpy2.robjects import pandas2ri
import anndata2ri

In [ ]:
import importlib
import warnings
warnings.filterwarnings("ignore")

import pickle as pkl

In [ ]:
# Ignore R warning messages
#Note: this can be commented out to get more verbose R output
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython

plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)

In [ ]:
%%R
# Load libraries from correct lib Paths for my environment - ignore this!
.libPaths(.libPaths('C:\\Users\\16220\\AppData\\Local\\R\\win-library\\4.2'))

# Load all the R libraries we will be using in the notebook
library(scran)
library(Seurat)
library(RColorBrewer)
library(slingshot)
library(monocle)
library(gam)
library(ggplot2)
library(plyr)
library(MAST)
library(clusterExperiment)
library(monocle3)

In [ ]:
cd45ra_neg= sc.read_10x_mtx("CD45RAneg", prefix = "GSE150132_aggregation_s1-5_",cache=True)

In [ ]:
cd45ra_neg.obs['n_counts'] = cd45ra_neg.X.sum(1)
cd45ra_neg.obs['log_counts'] = np.log(cd45ra_neg.obs['n_counts'])
cd45ra_neg.obs['n_genes'] = (cd45ra_neg.X > 0).sum(1)
cd45ra_neg.obs['log10GenesPerUMI'] = np.log10(cd45ra_neg.obs['n_genes'])/np.log10(cd45ra_neg.obs['n_counts']) #This is the complexity

mt_gene_mask = [gene.startswith('MT-') for gene in cd45ra_neg.var_names]
cd45ra_neg.obs['mt_frac'] = np.array(cd45ra_neg.X[:, mt_gene_mask].sum(1).ravel())[0]/cd45ra_neg.obs['n_counts']
ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in cd45ra_neg.var_names]
cd45ra_neg.obs['ribo_frac'] = np.array(cd45ra_neg.X[:, ribo_gene_mask].sum(1).ravel())[0]/cd45ra_neg.obs['n_counts']
print('Number of cells before T filter: {:d}'.format(cd45ra_neg.n_obs))
cd45ra_neg = cd45ra_neg[np.array(cd45ra_neg[:,'CD3D'].X.A>0)|np.array(cd45ra_neg[:,'CD3E'].X.A>0)|np.array(cd45ra_neg[:,'CD3G'].X.A>0),:]
cd45ra_neg = cd45ra_neg[np.array(cd45ra_neg[:,'CD19'].X.A==0),:]
print('Number of cells after T filter: {:d}'.format(cd45ra_neg.n_obs))

In [ ]:
# Quality control - plot QC metrics
t1 = sc.pl.violin(cd45ra_neg, 'n_counts', size=2, log=True, cut=0)
t2 = sc.pl.violin(cd45ra_neg, 'mt_frac')
t3 = sc.pl.violin(cd45ra_neg, 'ribo_frac')
t4 = sns.distplot(cd45ra_neg.obs['log10GenesPerUMI'],bins = 60)

In [ ]:
#Data quality summary plots
p1 = sc.pl.scatter(cd45ra_neg, 'n_counts', 'n_genes', color='mt_frac')
p2 = sc.pl.scatter(cd45ra_neg[cd45ra_neg.obs['n_counts']<10000], 'n_counts', 'n_genes', color='mt_frac')

In [ ]:
#Thresholding decision: counts
p3 = sns.distplot(cd45ra_neg.obs['n_counts'], kde=False)
plt.show()

p4 = sns.distplot(cd45ra_neg.obs['n_counts'][cd45ra_neg.obs['n_counts']<4000], kde=False, bins=60)
plt.show()

p5 = sns.distplot(cd45ra_neg.obs['n_counts'][cd45ra_neg.obs['n_counts']>10000], kde=False, bins=60)
plt.show()

In [ ]:
#Thresholding decision: genes
p6 = sns.distplot(cd45ra_neg.obs['n_genes'], kde=False, bins=60)
plt.show()

p7 = sns.distplot(cd45ra_neg.obs['n_genes'][cd45ra_neg.obs['n_genes']<1000], kde=False, bins=60)
plt.show()

In [ ]:
# Filter cells according to identified QC thresholds:
print('Total number of cells: {:d}'.format(cd45ra_neg.n_obs))

sc.pp.filter_cells(cd45ra_neg, min_counts = 0)
print('Number of cells after min count filter: {:d}'.format(cd45ra_neg.n_obs))

#sc.pp.filter_cells(cd45ra_neg, max_counts = 15000)
#print('Number of cells after max count filter: {:d}'.format(cd45ra_neg.n_obs))

cd45ra_neg = cd45ra_neg[cd45ra_neg.obs['mt_frac'] < 0.2]
print('Number of cells after MT filter: {:d}'.format(cd45ra_neg.n_obs))

cd45ra_neg = cd45ra_neg[cd45ra_neg.obs['ribo_frac'] > 0.05]
print('Number of cells after ribo filter: {:d}'.format(cd45ra_neg.n_obs))

sc.pp.filter_cells(cd45ra_neg, min_genes = 200)
print('Number of cells after gene filter: {:d}'.format(cd45ra_neg.n_obs))

#cd45ra_neg = cd45ra_neg[cd45ra_neg.obs['log10GenesPerUMI'] > 0.8]
#print('Number of cells after complexity filter: {:d}'.format(cd45ra_neg.n_obs))

In [ ]:
clf = doubletdetection.BoostClassifier(n_components = 50, n_top_var_genes = 3000)
# raw_counts is a cells by genes count matrix
labels = clf.fit(adata_group[1].X.A).predict()
# higher means more likely to be doublet
scores = clf.doublet_score()

In [ ]:
print('Number of cells before T filter: {:d}'.format(adata_group[1].n_obs))
adata_group[1] = adata_group[1][labels == 0]
print('Number of cells after T filter: {:d}'.format(adata_group[1].n_obs))

In [ ]:
sc.pp.filter_genes(cd45ra_neg, min_cells = 2)

In [ ]:
cd45ra_neg.obs['n_counts'] = cd45ra_neg.X.sum(1)
cd45ra_neg.obs['log_counts'] = np.log(cd45ra_neg.obs['n_counts'])
cd45ra_neg.obs['n_genes'] = (cd45ra_neg.X > 0).sum(1)
cd45ra_neg.obs['log10GenesPerUMI'] = np.log10(cd45ra_neg.obs['n_genes'])/np.log10(cd45ra_neg.obs['n_counts'])

mt_gene_mask = [gene.startswith('MT-') for gene in cd45ra_neg.var_names]
cd45ra_neg.obs['mt_frac'] = np.array(cd45ra_neg.X[:, mt_gene_mask].sum(1).ravel())[0]/cd45ra_neg.obs['n_counts']
ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in cd45ra_neg.var_names]
cd45ra_neg.obs['ribo_frac'] = np.array(cd45ra_neg.X[:, ribo_gene_mask].sum(1).ravel())[0]/cd45ra_neg.obs['n_counts']

In [ ]:
adata_forcycle = cd45ra_neg.copy()
sc.pp.normalize_total(adata_forcycle, key_added = 'normalization_factors')
sc.pp.log1p(adata_forcycle)

In [ ]:
sc.pp.highly_variable_genes(adata_forcycle, flavor='seurat', n_top_genes=3000)
print('\n','Number of highly variable genes: {:d}'.format(np.sum(adata_forcycle.var['highly_variable'])))

In [ ]:
sc.pp.pca(adata_forcycle, n_comps=50, use_highly_variable=True, svd_solver='arpack')
sc.pp.neighbors(adata_forcycle, n_neighbors = 15, n_pcs = 50)

In [ ]:
sc.tl.umap(adata_forcycle, min_dist = 0.4, spread = 1.2)

### Visualiaze different conditions, the batch effect is negligible

In [ ]:
sc.pl.umap(adata_forcycle, color='SELL')

### Use the cell cycle gene list summarized by Triosh et al. to calculate the cc score

In [ ]:
cc_genes = pd.read_excel('Triosh_cc.xlsx')

In [ ]:
s_genes = cc_genes['G1/S'].dropna()
g2m_genes = cc_genes['G2/M'].dropna()

s_genes_mm_ens = adata_forcycle.var_names[np.in1d(adata_forcycle.var_names, s_genes)]
g2m_genes_mm_ens = adata_forcycle.var_names[np.in1d(adata_forcycle.var_names, g2m_genes)]

In [ ]:
sc.tl.score_genes_cell_cycle(adata_forcycle, s_genes=s_genes_mm_ens, g2m_genes=g2m_genes_mm_ens)

### Visualize the cell phases of the co-cultured cells

In [ ]:
sc.pl.umap(adata_forcycle, color='phase',size = 8)

In [ ]:
Sscore = adata_forcycle.obs['S_score'].values.tolist()
G2Mscore = adata_forcycle.obs['G2M_score'].values.tolist()
S_G2M_diff = adata_forcycle.obs['S_score'] - adata_forcycle.obs['G2M_score']
S_G2M_diff = S_G2M_diff.values.tolist()

In [ ]:
cd45ra_neg.uns['allgenes'] = list(cd45ra_neg.var_names)
cd45ra_neg.obs['S_score'] = adata_forcycle.obs['S_score']
cd45ra_neg.obs['G2M_score'] = adata_forcycle.obs['G2M_score']
cd45ra_neg.obs['S_G2M_diff'] = cd45ra_neg.obs['S_score'] - cd45ra_neg.obs['G2M_score']
cd45ra_neg.obs['phase'] = adata_forcycle.obs['phase']

In [ ]:
cd45ra_neg.write('beforetrans_cd45ra_neg.h5ad')

In [ ]:
colnames = cd45ra_neg.obs_names
rownames = cd45ra_neg.var_names
mt_frac = cd45ra_neg.obs['mt_frac']
S_G2M_diff = cd45ra_neg.obs['S_score'] - cd45ra_neg.obs['G2M_score']
S_G2M_diff = S_G2M_diff.values.tolist()

### Construct the Seurat object

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(list(range(cd45ra_neg.shape[0])),test_size = 0.7)

In [ ]:
adata_train = cd45ra_neg[x_train,:]

In [ ]:
colnames = adata_train .obs_names
rownames = adata_train .var_names
mt_frac = adata_train .obs['mt_frac']
S_G2M_diff = adata_train .obs['S_score'] - adata_train .obs['G2M_score']
S_G2M_diff = S_G2M_diff .values.tolist()

In [ ]:
%%R -i adata_train -i colnames -i rownames -i S_G2M_diff -i mt_frac
counts = adata_train@assays@data$X
colnames(counts) = colnames
rownames(counts) = rownames

In [ ]:
%%R
srat <- CreateSeuratObject(counts = counts, project = "RNA", min.cells = 0, min.features = 0, assay = "RNA")
srat[["S_G2M_diff"]] = S_G2M_diff
srat[["mt_frac"]] = mt_frac

### SCTransform dataset separately

In [ ]:
%%R
srat = SCTransform(object = srat, verbose = FALSE, vst.flavor = "v2", vars.to.regress = c("S_G2M_diff","mt_frac"),min_cells=0) #c("G2Mscore","Sscore"))

In [ ]:
%%R
srat = RunPCA(srat, verbose = FALSE)

In [ ]:
%%R -o HVG -o varExplained -o PCs -o mat -o adata_transformed -o left_genes
HVG = VariableFeatures(srat)

mat <- Seurat::GetAssayData(object = srat, slot = 'scale.data')
pca <- srat[["pca"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))

eigValues = (pca@stdev)^2  ## EigenValues
varExplained = eigValues / total_variance

PCs = Loadings(srat, reduction = "pca")

adata_transformed = as.SingleCellExperiment(srat)
left_genes = rownames(x = srat[["SCT"]])

In [ ]:
%%R -o counts -o logcounts

counts = srat[["SCT"]]@counts
logcounts = srat[["SCT"]]@data

In [ ]:
%R -o srat

In [ ]:
import anndata
adata = anndata.AnnData(X = logcounts.T)
adata.var_names = left_genes
adata.obs_names = adata_transformed.obs_names
adata.obs = adata_train.obs
adata.uns['scaled'] = mat.T
adata.layers['counts'] = counts.T
adata.layers['logcounts'] = logcounts.T
adata.obsm['X_pca'] = adata_transformed.obsm['X_pca']

In [ ]:
adata.varm['PCs'] = np.zeros([np.shape(adata.X.A)[1],50])
for i in range(0,50):
    genename = HVG[i]
    adata.varm['PCs'][adata.var_names == genename,:] = PCs[i,:]

In [ ]:
adata.obs['n_counts'] = adata.layers['counts'].sum(1)
adata.obs['n_genes'] = (adata.layers['counts'] > 0).sum(1)
adata.var['highly_variable'] = adata.var_names.isin(HVG)
adata.obs['phase'] = adata_train.obs['phase']
adata.uns['pca'] = dict({'variance_ratio': varExplained})

In [ ]:
mt_gene_mask = [gene.startswith('MT-') for gene in adata.var_names]
adata.obs['mt_frac'] = np.array(adata.layers['counts'][:, mt_gene_mask].sum(1).ravel())[0]/adata.obs['n_counts']

In [ ]:
ribo_gene_mask = [gene.startswith('RPL') or gene.startswith('RPS') for gene in adata.var_names]
adata.obs['ribo_frac'] = np.array(adata.layers['counts'][:, ribo_gene_mask].sum(1).ravel())[0]/adata.obs['n_counts']

In [ ]:
sc.pp.neighbors(adata, n_pcs = 50)

In [ ]:
adata_copy = sc.tl.leiden(adata, resolution = 0.7,copy = True)
sc.pl.pca(adata_copy, color='leiden', legend_loc = 'on data')

In [ ]:
adata = adata_copy

In [ ]:
sc.tl.paga(adata, groups='leiden')
sc.pl.paga(adata,threshold = 0.2, labels=None, fontsize = 10, fontoutline = 3,node_size_scale = 3,node_size_power = 1,random_state = 0)

In [ ]:
sc.tl.umap(adata, min_dist = 0.01, spread = 2,init_pos = 'paga')

In [ ]:
plt.close()
plt.rcParams['axes.linewidth'] = 2
fig = sc.pl.umap(adata, color = ['CD8A','leiden'],size = 50, legend_loc = 'on data', palette  = my_palette, ncols = 4, cmap = my_cmap, return_fig = True, legend_fontsize = 'large')
ax = fig.get_axes()
for i in range(0,len(ax)):
    ax[i].xaxis.label.set_fontsize(22)
    ax[i].xaxis.label.set_fontweight('bold')
    ax[i].yaxis.label.set_fontsize(22)
    ax[i].title.set_fontsize(30)
    ax[i].yaxis.label.set_fontweight('bold')
    ax[i].title.set_fontweight('bold')
#plt.savefig('overall_map_withunmatched.png')

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
#This pallete is for colorblinds
my_palette = ['#0351A8','#8CB0E0','#D56D11','#FFBB78','#234E08','#53CB8B','#D30083','#CB788D','#4E195A','#C58CCF','#AA290F','#B03FD1','#E8BCCF','#64605F','#B2AD9A','#D2D30B','#D1BD4F','#06DCF2','#9EDAE5','#517219','#5B43CF','#D92F24','#FFD900','#002F33','#B8A3A3']

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
values = [0,1]
colors = [(227, 227, 227), (255, 42, 18)]
norm = plt.Normalize(min(values), max(values))
my_cmap = LinearSegmentedColormap.from_list(
    '', [(norm(value), tuple(np.array(color) / 255)) for value, color in zip(values, colors)])

In [ ]:
adata.write('GSE150132_processed.h5ad')